# FINANCIAL COMPLAINT TOPIC MODELING - NMF

### Albert Opoku - Senior Statistical Consultant at Allianca data Inc

#### Contact me:
- twitter [@opalbert](https://twitter.com/opalbert)
- linkedIn [Albert Opoku](https://www.linkedin.com/in/albertopokupmachinelearning/)
- email opalkabert@gmail.com
- website [www.opokualbert.com](https://opokualbert.com/)

### CASE:

Consumers provide feedback on financial products or services and our task is to extract the hidden themes/topics and assign each of the feedback documents to one of these themes or topics.

### Solution:

Train a Natural Language Processing machine learning model to extract the topics from each of the open-ended complaint text document.

### Data Source:

The data is downloaded from kaggle via this url: [consumer complaint data](https://www.kaggle.com/cfpb/us-consumer-finance-complaints)








**Topic Modeling** is an unsupervized machine learning technique to discover the hidden/latent thematic structure in a large corpus of text documents.


[Latent Dirichlet Analysis (LDA)](http://jmlr.org/papers/volume3/blei03a/blei03a.pdf) and [Non-Negative Matrix Fatactorization (NMF)](https://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) are the two most popular topic modeling techniques. LDA uses a probabilistic approach where as NMF uses matrix factorization approach.


<img src="NMF_Equation.PNG">

<img src="DTM_.png">

<img src="Doc_Topic_Terms.png">

##### Import the needed packages

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.decomposition import NMF
import numpy as np
import pickle 
import warnings  
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

import nltk 
# from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
# from nltk.stem import PorterStemmer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df_orig = pd.read_csv("consumer_complaints.csv")

In [3]:
df_orig.shape

(555957, 18)

In [4]:
df_orig.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [5]:
# Check how many rows have missing values for consumer_complaint_narrative column
df_orig.consumer_complaint_narrative.isnull().values.sum()

489151

In [6]:
# extract only the relevant column for this project
df_orig = df_orig.loc[:,['consumer_complaint_narrative']]
# df_orig.to_csv('consumer_complaint_text.csv', index=False)

In [7]:
# Exclude all rows with null consumer_complaint_narrative
df_orig.dropna(inplace=True)
df_orig.shape

(66806, 1)

In [8]:
pd.set_option('max_colwidth', 1000)
pd.options.display.max_rows = 500

In [9]:
# Create a doc id for merging the results back to the original file 
df_orig.insert(0, 'Doc_Id', range(0, 0 + len(df_orig)))
df_orig.head()

,Doc_Id,consumer_complaint_narrative
190126,0,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
190135,1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X..."
190155,2,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b..."
190207,3,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n"
190208,4,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar..."


In [10]:
# save for use later
df_orig.to_pickle('df_orig.pkl')

In [11]:
df = df_orig.loc[:,['consumer_complaint_narrative','Doc_Id']]

In [12]:
# Get the word count for each document
df['word_count'] = df['consumer_complaint_narrative'].apply(lambda x: len(str(x).split(" ")))
df.head()

,consumer_complaint_narrative,Doc_Id,word_count
190126,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n,0,56
190135,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X...",1,249
190155,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b...",2,667
190207,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n",3,76
190208,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar...",4,260


In [13]:
# Summary statistics
df.word_count.describe()

count    66806.000000
mean       191.662830
std        166.875112
min          2.000000
25%         72.000000
50%        137.000000
75%        255.000000
max       1285.000000
Name: word_count, dtype: float64

In [14]:
# just to get a sizable data to work with in this tutorial
df1= df[(df['word_count']>=191)&(df['word_count']<=255)]

In [15]:
df1.shape

(7491, 3)

In [16]:
df1.head()

,consumer_complaint_narrative,Doc_Id,word_count
190135,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X...",1,249
191998,"Around XXXX we consolidated our loans and paid off a loan to Chrysler financial for an automotive loan. In XX/XX/XXXX we applied for a lost title at our local town office in Tennessee. At this time it was made known to us that TD Financial XXXX XXXX XXXX XXXX XXXX lien on our title. I was directed to TD financial XXXX XXXX, then to My title Support XXXX, and back to TD financial to find that there is a "" balance due of {$85.00} '' for "" charges and fees ''. I have NEVER received any notifications from XXXX, TD Auto Finance or any other representative in the TEN to TWELVE YEARS since that loan was paid off. TD Auto Finance is located in XXXX Maine and I have never transacted business with them. I requested a supervisor call me to discuss the "" balance due '' on XXXX/XXXX/15, and 1 week has passed with no response despite the XXXX calls I have placed to their "" customer service department '' I find it very concerning and unethical that a corporation can quietly hold a "" balance due '...",35,201
192083,"I was offered a trial modification by my servicer - Carrington. I accepted the offer. The first payment was due in the month of XXXX. As a member of the XXXX XXXX XXXX XXXX, I received 30 day XXXX in the month of XXXX. None-the-less I called Carrington to make the XXXX trial modification payment over the phone. Carrington refused to take the payment over the phone but instead demanded that I send via XXXX XXXX. Because of my service I explained to Carrington that I would have limited access to XXXX XXXX and I faxed my XXXX. Carrington stood firm in their unreasonableness. I was able to get to a XXXX XXXX office on XXXX XXXX, but Carrington refused to accept the payment because they claimed it was late. My Authorized XXXX Party representative called Carrington on XXXX separate occasions - XXXX XXXX, XXXX XXXX, and XXXX XXXX, 2015. The purpose of each call was to determine whether or not there was a scheduled foreclosure sale date. Each time the Authorized XXXX Party Representative w...",45,210
192137,"Due to financial hardship, I have been unable to pay on XXXX of my student loans. I have been struggling for months and have contacted the lender via email and written letters. The only response I receive is that I must contact via phone between certain hours. I have told them multiple times that is impossible due to my work schedule. They have written and offered to settle or make arrangements and when I write them back they always state they can not help unless I CALL them. It 's as if they are purposefully contributing to the default my account has now entered. They last letter I received states that default will occur on XXXX XXXX, if I do not contact them. I checked my credit report and they had already reported default and placed with a collection agency ( 2 weeks prior to their date ). I emailed them regarding the situation and still they will not discuss anything in writi

In [17]:
# Removing unwanted characters
import re
df1['consumer_complaint_narrative'] = df1['consumer_complaint_narrative'].str.replace('X', '')
df1['consumer_complaint_narrative'] = df1['consumer_complaint_narrative'].str.replace('{', '')
df1['consumer_complaint_narrative'] = df1['consumer_complaint_narrative'].str.replace('}', '')
df1['consumer_complaint_narrative'] = df1['consumer_complaint_narrative'].str.replace('/', '')
df1.head()

,consumer_complaint_narrative,Doc_Id,word_count
190135,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in 2015, per my credit monitoring agency, it showed a delinquent balance of $0.00, but when I checked my credit report again on 2015, ther...",1,249
191998,"Around we consolidated our loans and paid off a loan to Chrysler financial for an automotive loan. In we applied for a lost title at our local town office in Tennessee. At this time it was made known to us that TD Financial lien on our title. I was directed to TD financial , then to My title Support , and back to TD financial to find that there is a "" balance due of $85.00 '' for "" charges and fees ''. I have NEVER received any notifications from , TD Auto Finance or any other representative in the TEN to TWELVE YEARS since that loan was paid off. TD Auto Finance is located in Maine and I have never transacted business with them. I requested a supervisor call me to discuss the "" balance due '' on 15, and 1 week has passed with no response despite the calls I have placed to their "" customer service department '' I find it very concerning and unethical that a corporation can quietly hold a "" balance due '' for more than 10 years and then demand a consumer pay. \n",35,201
192083,"I was offered a trial modification by my servicer - Carrington. I accepted the offer. The first payment was due in the month of . As a member of the , I received 30 day in the month of . None-the-less I called Carrington to make the trial modification payment over the phone. Carrington refused to take the payment over the phone but instead demanded that I send via . Because of my service I explained to Carrington that I would have limited access to and I faxed my . Carrington stood firm in their unreasonableness. I was able to get to a office on , but Carrington refused to accept the payment because they claimed it was late. My Authorized Party representative called Carrington on separate occasions - , , and , 2015. The purpose of each call was to determine whether or not there was a scheduled foreclosure sale date. Each time the Authorized Party Representative was told there was not a foreclosure sale date. Carrington completed a foreclosure sale on , 2015. At th...",45,210
192137,"Due to financial hardship, I have been unable to pay on of my student loans. I have been struggling for months and have contacted the lender via email and written letters. The only response I receive is that I must contact via phone between certain hours. I have told them multiple times that is impossible due to my work schedule. They have written and offered to settle or make arrangements and when I write them back they always state they can not help unless I CALL them. It 's as if they are purposefully contributing to the default my account has now entered. They last letter I received states that default will occur on , if I do not contact them. I checked my credit report and they had already reported default and placed with a collection agency ( 2 weeks prior to their date ). I emailed them regarding the situation and still they will not discuss anything in writing ONLY by phone. This makes no sense to me. \nI can not co

In [18]:
# Top 20 most frequent words
freq = pd.Series(' '.join(df1['consumer_complaint_narrative']).split()).value_counts()[:20]
freq

the       61969
I         61663
to        55356
and       42990
a         33372
my        31148
of        24544
that      24332
was       21922
they      17731
in        17470
have      17187
on        16073
not       15818
for       15762
is        15324
me        13606
with      13230
this      12806
credit    10081
dtype: int64

In [19]:
# Work with only nouns
def nouns(text):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [20]:
df1['data_nouns'] = pd.DataFrame(df1.consumer_complaint_narrative.apply(nouns))
df1.head()

,consumer_complaint_narrative,Doc_Id,word_count,data_nouns
190135,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in 2015, per my credit monitoring agency, it showed a delinquent balance of $0.00, but when I checked my credit report again on 2015, ther...",1,249,inconsistencies amount M T Bank amount credit reporting agencies letter order issue entry credit report vehicle loss insurance company insurance company GAP insurancw balance loan M T Bank balance loan faith proof balance payment debt goodwill letter remainder debt exchange removal entry credit report point credit monitoring agency balance credit report balance monies GAP insurance insurance company M T Bank M T Bank return mail courtesy response
191998,"Around we consolidated our loans and paid off a loan to Chrysler financial for an automotive loan. In we applied for a lost title at our local town office in Tennessee. At this time it was made known to us that TD Financial lien on our title. I was directed to TD financial , then to My title Support , and back to TD financial to find that there is a "" balance due of $85.00 '' for "" charges and fees ''. I have NEVER received any notifications from , TD Auto Finance or any other representative in the TEN to TWELVE YEARS since that loan was paid off. TD Auto Finance is located in Maine and I have never transacted business with them. I requested a supervisor call me to discuss the "" balance due '' on 15, and 1 week has passed with no response despite the calls I have placed to their "" customer service department '' I find it very concerning and unethical that a corporation can quietly hold a "" balance due '' for more than 10 years and then demand a consumer pay. \n",35,201,loans loan Chrysler loan title town office Tennessee time TD Financial lien title TD My title Support TD balance charges fees NEVER notifications TD Auto Finance representative TEN TWELVE YEARS loan TD Auto Finance Maine business supervisor call balance week response calls customer service department corporation balance years consumer pay
192083,"I was offered a trial modification by my servicer - Carrington. I accepted the offer. The first payment was due in the month of . As a member of the , I received 30 day in the month of . None-the-less I called Carrington to make the trial modification payment over the phone. Carrington refused to take the payment over the phone but instead demanded that I send via . Because of my service I explained to Carrington that I would have limited access to and I faxed my . Carrington stood firm in their unreasonableness. I was able to get to a office on , but Carrington refused to accept the payment because they claimed it was late. My Authorized Party representative called Carrington on separate occasions - , , and , 2015. The purpose of each call was to determine whether or not there was a scheduled foreclosure sale date. Each time the Authorized Party Representative was told there was not a foreclosure sale date. Carrington completed a foreclosure sale on , 2015. At th...",45,210,trial modification servicer Carrington offer payment month member day month None-the-less Carrington trial modification payment phone Carringt

In [21]:
# Further cleaning, removing stopwords, lemmatizing
import re
temp =[]
my_stop_words = text.ENGLISH_STOP_WORDS

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

for sentence in df1['data_nouns']:
    sentence = sentence.lower()
    cleaner = re.compile('<.*?>')
    sentence = re.sub(cleaner, ' ', sentence)  # Remove html tags
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence) # removing puntuations
    
    words = [lemmatizer.lemmatize(word) for word in sentence.split() if word not in my_stop_words] # removing stopwords and lemm
    temp.append(words)
    
final_X = temp

In [22]:
final_X[:2]

[['inconsistency',
  'm',
  't',
  'bank',
  'credit',
  'reporting',
  'agency',
  'letter',
  'order',
  'issue',
  'entry',
  'credit',
  'report',
  'vehicle',
  'loss',
  'insurance',
  'company',
  'insurance',
  'company',
  'gap',
  'insurancw',
  'balance',
  'loan',
  'm',
  't',
  'bank',
  'balance',
  'loan',
  'faith',
  'proof',
  'balance',
  'payment',
  'debt',
  'goodwill',
  'letter',
  'remainder',
  'debt',
  'exchange',
  'removal',
  'entry',
  'credit',
  'report',
  'point',
  'credit',
  'monitoring',
  'agency',
  'balance',
  'credit',
  'report',
  'balance',
  'monies',
  'gap',
  'insurance',
  'insurance',
  'company',
  'm',
  't',
  'bank',
  'm',
  't',
  'bank',
  'return',
  'mail',
  'courtesy',
  'response'],
 ['loan',
  'loan',
  'chrysler',
  'loan',
  'title',
  'town',
  'office',
  'tennessee',
  'time',
  'td',
  'financial',
  'lien',
  'title',
  'td',
  'title',
  'support',
  'td',
  'balance',
  'charge',
  'fee',
  'notification',
  '

In [23]:
sent = []
for row in final_X:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)
final_X = sent

In [24]:
print(final_X[:2])

[' inconsistency m t bank credit reporting agency letter order issue entry credit report vehicle loss insurance company insurance company gap insurancw balance loan m t bank balance loan faith proof balance payment debt goodwill letter remainder debt exchange removal entry credit report point credit monitoring agency balance credit report balance monies gap insurance insurance company m t bank m t bank return mail courtesy response', ' loan loan chrysler loan title town office tennessee time td financial lien title td title support td balance charge fee notification td auto finance representative year loan td auto finance maine business supervisor balance week response call customer service department corporation balance year consumer pay']


In [25]:
# remove unwanted characters, numbers and symbols 
df1['cleaned'] = final_X
df1.head()

,consumer_complaint_narrative,Doc_Id,word_count,data_nouns,cleaned
190135,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in 2015, per my credit monitoring agency, it showed a delinquent balance of $0.00, but when I checked my credit report again on 2015, ther...",1,249,inconsistencies amount M T Bank amount credit reporting agencies letter order issue entry credit report vehicle loss insurance company insurance company GAP insurancw balance loan M T Bank balance loan faith proof balance payment debt goodwill letter remainder debt exchange removal entry credit report point credit monitoring agency balance credit report balance monies GAP insurance insurance company M T Bank M T Bank return mail courtesy response,inconsistency m t bank credit reporting agency letter order issue entry credit report vehicle loss insurance company insurance company gap insurancw balance loan m t bank balance loan faith proof balance payment debt goodwill letter remainder debt exchange removal entry credit report point credit monitoring agency balance credit report balance monies gap insurance insurance company m t bank m t bank return mail courtesy response
191998,"Around we consolidated our loans and paid off a loan to Chrysler financial for an automotive loan. In we applied for a lost title at our local town office in Tennessee. At this time it was made known to us that TD Financial lien on our title. I was directed to TD financial , then to My title Support , and back to TD financial to find that there is a "" balance due of $85.00 '' for "" charges and fees ''. I have NEVER received any notifications from , TD Auto Finance or any other representative in the TEN to TWELVE YEARS since that loan was paid off. TD Auto Finance is located in Maine and I have never transacted business with them. I requested a supervisor call me to discuss the "" balance due '' on 15, and 1 week has passed with no response despite the calls I have placed to their "" customer service department '' I find it very concerning and unethical that a corporation can quietly hold a "" balance due '' for more than 10 years and then demand a consumer pay. \n",35,201,loans loan Chrysler loan title town office Tennessee time TD Financial lien title TD My title Support TD balance charges fees NEVER notifications TD Auto Finance representative TEN TWELVE YEARS loan TD Auto Finance Maine business supervisor call balance week response calls customer service department corporation balance years consumer pay,loan loan chrysler loan title town office tennessee time td financial lien title td title support td balance charge fee notification td auto finance representative year loan td auto finance maine business supervisor balance week response call customer service department corporation balance year consumer pay
192083,"I was offered a trial modification by my servicer - Carrington. I accepted the offer. The first payment was due in the month of . As a member of the , I received 30 day in the month of . None-the-less I called Carrington to make the trial modification payment over the phone. Carrington refused to take the payment over the phone but instead demanded that I send via . Because of my

In [26]:
# save for use later
df1.to_pickle('data_prep.pkl')

In [27]:
#df1 = pd.read_pickle(data_prep.pkl)